# Data Prep
Our dataset already has an appropraite file structure with class divided within each of the folder (train/test).  
In this document, we will just create a validation set.

In [6]:
import os, shutil
import numpy as np

In [4]:
# make necessary directories

def makedirect(paths):
    for path in paths:
        try: os.mkdir(path)
        except FileExistsError: 
            print(f'{path} already exists')

makedirect(['DATA/validation', 'DATA/validation/NORMAL', 'DATA/validation/PNEUMONIA'])

In [11]:
# pull images from training into validation sets
normal_imgs = [fn for fn in os.listdir('DATA/train/NORMAL') if fn.endswith('.jpeg')]
pneumo_imgs = [fn for fn in os.listdir('DATA/train/PNEUMONIA') if fn.endswith('.jpeg')]

# randomly select 20% of it 
normal_val = np.random.choice(normal_imgs, int(.2 * len(normal_imgs)), replace = False)
pneumo_val = np.random.choice(pneumo_imgs, int(.2 * len(pneumo_imgs)), replace = False)   


In [13]:
# move files
def move_files(olddir, newdir, fnames):
    for fn in fnames:
        shutil.move(f'{olddir}/{fn}', f'{newdir}/{fn}')

move_files('DATA/train/NORMAL', 'DATA/validation/NORMAL', normal_val)
move_files('DATA/train/PNEUMONIA', 'DATA/validation/PNEUMONIA', pneumo_val)